# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [3]:
import openai
from openai import OpenAI
import pandas as pd

# Set up your OpenAI API key
api_key = ""

client = OpenAI(api_key=api_key)

In [4]:
## Create the assistant
assistant = client.beta.assistants.create(
    name="OpenAI Assistant",
    instructions="You are an expert in designing curriculum vitae. Please provide detailed, professional, and clear instructions on creating an impressive CV",
    model="gpt-4o-mini")

In [5]:
# Create a vector store called "CV Documents"
vector_store = client.beta.vector_stores.create(name="CV Documents")

In [10]:
# Ready the files for upload to OpenAI
file_paths = ["../CV-Guide.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

In [8]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [9]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [11]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Write a CV introduction for a junior data scientist position",
    }
  ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
print(message_content.value)

Certainly! Here’s a sample CV introduction for a junior data scientist position:

---

**Introduction**

Enthusiastic and analytical junior data scientist with a strong foundation in statistical analysis, machine learning, and data visualization. Holding a degree in [Your Degree] from [Your University], I have developed a solid understanding of data-driven decision-making through academic projects and internships. Proficient in programming languages such as Python and R, and experienced with tools like SQL and Tableau, I possess a passion for converting complex data into actionable insights. Eager to contribute to a dynamic team where I can further develop my skills and help drive business growth through data analytics.

---

Feel free to customize the introduction with your specific degree, university, and any relevant experiences or skills!


## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [12]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
print(forecast)

{'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.03802776336669922, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2024-10-29T00:00', '2024-10-29T01:00', '2024-10-29T02:00', '2024-10-29T03:00', '2024-10-29T04:00', '2024-10-29T05:00', '2024-10-29T06:00', '2024-10-29T07:00', '2024-10-29T08:00', '2024-10-29T09:00', '2024-10-29T10:00', '2024-10-29T11:00', '2024-10-29T12:00', '2024-10-29T13:00', '2024-10-29T14:00', '2024-10-29T15:00', '2024-10-29T16:00', '2024-10-29T17:00', '2024-10-29T18:00', '2024-10-29T19:00', '2024-10-29T20:00', '2024-10-29T21:00', '2024-10-29T22:00', '2024-10-29T23:00', '2024-10-30T00:00', '2024-10-30T01:00', '2024-10-30T02:00', '2024-10-30T03:00', '2024-10-30T04:00', '2024-10-30T05:00', '2024-10-30T06:00', '2024-10-30T07:00', '2024-10-30T08:00', '2024-10-30T09:00', '2024-10-30T10:00', '2024-10-30T11:00', '2024-10-30T12:00'

In [13]:
user_input = input('Enter the city in which you want to know the weather')

In [14]:
prompt_weather= f"""
    #Question: What is the weather in {user_input}
    #Role
    You are a helpful assistant for users asking about the weather.

    #Context
    You are using an API (open-meteo), which expects the longitude and latitude, in order to return the weather.
    
    #Chain of thought
    Here are the steps you need to follow to complete this task:

        1. 'First, you need to retrieve the longitud of the city in {user_input}'
        2. 'Next, return the information in json.object format'

    #Example:
    This is an example. 
        -Question: 'What is the weather in Madrid?. 
        -Asnwer: 'city': 'Madrid', 'coordinates': 'latitude': 40.4168, 'longitude': -3.7038' in json format

    What are the subsequent steps they need to take to submit their application?"

    User Request: Weather in {user_input}
    """

In [15]:
def weather_assistant(prompt):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
        )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id, assistant_id='asst_Ii10AeQ9FGu5Rwfexd4PWb0l'
    ) 

    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    message_json = json.loads(message_content.value)

    coordinates = message_json['coordinates']
    return get_weather_forecast(coordinates['latitude'], coordinates['longitude'])

weather_assistant(prompt=prompt_weather)

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'asst_Ii10AeQ9FGu5Rwfexd4PWb0l'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality